In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from dobletau_quant import BotConnector

In [2]:
# ------------------------------------
# Parámetros de fecha 
# ------------------------------------
start_date = pd.to_datetime("2024-01-20").date()  # Cambia según sea necesario
end_date = pd.to_datetime("2025-03-07").date()

# ------------------------------------
# Definición de tus tickers del portafolio
# ------------------------------------
portfolio_tickers = [
    "AAPL", "JPM", "NFLX", "CSCO", "GE", "DHR", "ANET", "UNP", "FI", "VRTX",
    "NVDA", "LLY", "BAC", "MS", "AMD", "INTU", "C", "UBER", "MU", "MRVL",
    "MSFT", "V", "JNJ", "NOW", "ABT", "PLTR", "PFE", "CMCSA", "LMT", "SBUX",
    "GOOG", "XOM", "CRM", "AXP", "GS", "VZ", "LOW", "KKR", "PANW", "MMC",
    "AMZN", "MA", "ABBV", "BX", "DIS", "BKNG", "AMGN", "SCHW", "APP", "NKE",
    "META", "UNH", "CVX", "TMO", "PM", "RTX", "SYK", "TJX", "GILD", "PLD",
    "TSLA", "ORCL", "KO", "ISRG", "ADBE", "T", "NEE", "COP", "BMY", "RLCX",
    "AVGO", "COST", "WFC", "IBM", "CAT", "AMAT", "BSX", "BA", "UPS", "KLAC",
    "BRK-B", "HD", "TMUS", "PEP", "QCOM", "SPGI", "HON", "DE", "GEV", "CHTR",
    "WMT", "PG", "MRK", "MCD", "TXN", "BLK", "PGR", "ADP", "ADI", "CEG"
]

In [3]:

# ------------------------------------
# Función para obtener datos de Alpha Vantage
# ------------------------------------
MY_API_KEY = "I8O7BK7GFH4O0HTK"

def fetch_data_alphavantage(ticker, api_key, outputsize='compact'):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "apikey": api_key,
        "outputsize": outputsize
    }
    
    try:
        response = requests.get(base_url, params=params)
        data_json = response.json()
        
        if "Time Series (Daily)" not in data_json:
            print(f"⚠️ No se encontró 'Time Series (Daily)' para {ticker}. Respuesta: {data_json}")
            return None
        
        daily_data = data_json["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(daily_data, orient="index", dtype=float)
        df.rename(columns={
            "1. open": "Open",
            "2. high": "High",
            "3. low": "Low",
            "4. close": "Close",
            "5. volume": "Volume"
        }, inplace=True)
        df.index = pd.to_datetime(df.index)
        df.sort_index(inplace=True)
        df = df[(df.index.date >= start_date) & (df.index.date <= end_date)]
        return df
    
    except Exception as e:
        print(f"❌ Error al obtener datos de {ticker}: {e}")
        return None

In [4]:

# ------------------------------------
# Descarga de datos para cada ticker
# ------------------------------------
raw_portfolio_data = {}  # Datos originales (raw)
print("Iniciando descarga de datos de Alpha Vantage...")

for ticker in portfolio_tickers:
    print(f"Descargando datos de {ticker}...")
    df_ticker = fetch_data_alphavantage(ticker, MY_API_KEY)
    if df_ticker is not None and not df_ticker.empty:
        raw_portfolio_data[ticker] = df_ticker
    else:
        print(f"Sin datos para {ticker}, se omite.")

Iniciando descarga de datos de Alpha Vantage...
Descargando datos de AAPL...
Descargando datos de JPM...
Descargando datos de NFLX...
Descargando datos de CSCO...
Descargando datos de GE...
Descargando datos de DHR...
Descargando datos de ANET...
Descargando datos de UNP...
Descargando datos de FI...
Descargando datos de VRTX...
Descargando datos de NVDA...
Descargando datos de LLY...
Descargando datos de BAC...
Descargando datos de MS...
Descargando datos de AMD...
Descargando datos de INTU...
Descargando datos de C...
Descargando datos de UBER...
Descargando datos de MU...
Descargando datos de MRVL...
Descargando datos de MSFT...
Descargando datos de V...
Descargando datos de JNJ...
Descargando datos de NOW...
Descargando datos de ABT...
Descargando datos de PLTR...
⚠️ No se encontró 'Time Series (Daily)' para PLTR. Respuesta: {'Note': 'We have detected your API key as I8O7BK7GFH4O0HTK and our standard API rate limit is 25 requests per day. Please visit https://www.alphavantage.co/pr

In [5]:
# ------------------------------------
# Verificación de actualización de datos
# ------------------------------------
print(f"\nVerificando actualización de datos (hasta {end_date}):")
portfolio_data = {}  # Este será el diccionario que usaremos para análisis

for ticker, df in raw_portfolio_data.items():
    # Verificamos si la última fecha del DataFrame es >= end_date
    if df.index.max().date() >= end_date:
        portfolio_data[ticker] = df
    else:
        print(f"⚠️ {ticker} tiene datos hasta {df.index.max().date()}, no actualizado a {end_date}.")


Verificando actualización de datos (hasta 2025-03-07):


In [6]:

# ------------------------------------
# Funciones de análisis técnico
# ------------------------------------
def fractal_pivots(df, n=3):
    """
    Detecta pivotes altos y bajos en función de una ventana de tamaño 2*n+1.
    """
    df["PivotHigh"] = np.nan
    df["PivotLow"] = np.nan
    for i in range(n, len(df) - n):
        high_window = df["High"].iloc[i - n: i + n + 1]
        low_window = df["Low"].iloc[i - n: i + n + 1]
        if df["High"].iloc[i] == high_window.max():
            df.at[df.index[i], "PivotHigh"] = df["High"].iloc[i]
        if df["Low"].iloc[i] == low_window.min():
            df.at[df.index[i], "PivotLow"] = df["Low"].iloc[i]
    return df

def zigzag_indicator(df, zigzag_threshold=0.01):
    """
    Genera el indicador ZigZag basado en cambios porcentuales a partir de los pivotes detectados.
    """
    df["ZigZag"] = np.nan
    last_pivot = None  
    direction = None  

    for i in range(len(df)):
        if not pd.isna(df["PivotHigh"].iloc[i]) or not pd.isna(df["PivotLow"].iloc[i]):
            current_price = float(df["Close"].iloc[i])
            if last_pivot is None:
                last_pivot = current_price
                df.at[df.index[i], "ZigZag"] = current_price
            else:
                change = (current_price - last_pivot) / last_pivot

                if direction is None:
                    if abs(change) >= zigzag_threshold:
                        direction = "up" if change > 0 else "down"
                        last_pivot = current_price
                        df.at[df.index[i], "ZigZag"] = current_price
                else:
                    if direction == "up" and current_price < last_pivot * (1 - zigzag_threshold):
                        direction = "down"
                        last_pivot = current_price
                        df.at[df.index[i], "ZigZag"] = current_price
                    elif direction == "down" and current_price > last_pivot * (1 + zigzag_threshold):
                        direction = "up"
                        last_pivot = current_price
                        df.at[df.index[i], "ZigZag"] = current_price

    return df

def compute_fib_levels(df):
    """
    Calcula los niveles de Fibonacci basados en los últimos pivotes altos y bajos detectados.
    Se crean columnas para los niveles de Fibonacci y se actualizan conforme se detectan nuevos pivotes.
    """
    # Inicializamos las columnas de Fibonacci
    df["Fib_0.236"] = np.nan
    df["Fib_0.382"] = np.nan
    df["Fib_0.500"] = np.nan
    df["Fib_0.618"] = np.nan
    df["Fib_0.786"] = np.nan

    last_pivot_high = None
    last_pivot_low = None

    for i in range(len(df)):
        # Actualizamos los pivotes si existen en la fila actual
        if not pd.isna(df["PivotHigh"].iloc[i]):
            last_pivot_high = df["PivotHigh"].iloc[i]
        if not pd.isna(df["PivotLow"].iloc[i]):
            last_pivot_low = df["PivotLow"].iloc[i]
        
        # Solo se calculan los niveles si se han identificado ambos pivotes
        if last_pivot_high is not None and last_pivot_low is not None:
            # Definir el swing: el valor más alto se toma como swing_high y el más bajo como swing_low
            if last_pivot_high > last_pivot_low:
                swing_high = last_pivot_high
                swing_low = last_pivot_low
            else:
                swing_high = last_pivot_low
                swing_low = last_pivot_high
            
            diff = swing_high - swing_low
            df.at[df.index[i], "Fib_0.236"] = swing_high - 0.236 * diff
            df.at[df.index[i], "Fib_0.382"] = swing_high - 0.382 * diff
            df.at[df.index[i], "Fib_0.500"] = swing_high - 0.500 * diff
            df.at[df.index[i], "Fib_0.618"] = swing_high - 0.618 * diff
            df.at[df.index[i], "Fib_0.786"] = swing_high - 0.786 * diff

    return df

def generate_signals(df):
    """
    Función de ejemplo para generar señales de trading basadas en el cruce del precio con niveles de Fibonacci.
    Las condiciones se pueden ajustar según la estrategia deseada.
    """
    df["Signal"] = np.nan
    for i in range(1, len(df)):
        # Ejemplo de señal de compra: el precio cierra por debajo y luego cruza al alza el nivel Fib_0.618
        if pd.notna(df["Fib_0.618"].iloc[i-1]) and pd.notna(df["Fib_0.618"].iloc[i]):
            if df["Close"].iloc[i-1] < df["Fib_0.618"].iloc[i-1] and df["Close"].iloc[i] > df["Fib_0.618"].iloc[i]:
                df.at[df.index[i], "Signal"] = "Buy"
            # Ejemplo de señal de venta: el precio cierra por encima y luego cruza a la baja el nivel Fib_0.382
            elif df["Close"].iloc[i-1] > df["Fib_0.382"].iloc[i-1] and df["Close"].iloc[i] < df["Fib_0.382"].iloc[i]:
                df.at[df.index[i], "Signal"] = "Sell"
    return df




In [7]:
# ------------------------------------
# Análisis técnico para todos los tickers
# ------------------------------------
for ticker, df in portfolio_data.items():
    df = fractal_pivots(df)
    df = zigzag_indicator(df)
    df = compute_fib_levels(df)
    df = generate_signals(df)
    # Guardamos de nuevo en el mismo diccionario
    portfolio_data[ticker] = df

C:\Users\diego\AppData\Local\Temp\ipykernel_43804\2222722059.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Buy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[df.index[i], "Signal"] = "Buy"
C:\Users\diego\AppData\Local\Temp\ipykernel_43804\2222722059.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Buy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[df.index[i], "Signal"] = "Buy"
C:\Users\diego\AppData\Local\Temp\ipykernel_43804\2222722059.py:107: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Sell' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[df.index[i], "Signal"] = "Sell"
C:\Users\diego\AppData\Local\Temp\ipyk

In [8]:
# ------------------------------------
# Dynamic Risk Management Function
# ------------------------------------
def dynamic_risk_management(capital, portfolio_value, max_risk=0.02, volatility=0.05):
    """
    Adjusts trade size dynamically based on portfolio performance and volatility.

    - Reduces position size after drawdowns.
    - Increases position size when portfolio is growing.
    - Uses volatility as a trade size adjustment factor.

    Parameters:
    - capital: Current available capital ($).
    - portfolio_value: Total portfolio value ($).
    - max_risk: Maximum % of capital to risk per trade.
    - volatility: Estimated market volatility (default 5%).

    Returns:
    - Adjusted trade size ($).
    """
    risk_factor = max_risk * (portfolio_value / capital)
    adjusted_trade_size = risk_factor * capital * (1 - volatility)
    return max(0, adjusted_trade_size)  # Ensure trade size is never negative


In [9]:
from dobletau_quant import BotConnector

# Inicializa el cliente con tu token único
bot = BotConnector(token="1ae620573a5c2c01")

info = bot.get_balance()
efectivo = info["Disponible"]
print(efectivo)

Información de la cuenta obtenida exitosamente.
88823.74


In [10]:
def calculate_orders(portfolio_data, capital=efectivo, risk_per_trade=0.02):
    """
    Generates a final list of orders for all tickers (buy, sell, or hold).
    
    - Uses Fibonacci retracements to determine buy/sell signals.
    - Ensures every ticker has an order (even if it's a hold).
    - Implements dynamic position sizing based on available capital.

    Parameters:
    - portfolio_data: Dictionary with stock data.
    - capital: Available capital ($).
    - risk_per_trade: % of capital to risk per trade.

    Returns:
    - orders: List of tuples [(ticker, qty)] (qty > 0 for buy, < 0 for sell, 0 for hold)
    """
    orders = []  # List to store orders

    for ticker, df in portfolio_data.items():
        last_row = df.iloc[-1]  # Get last row for signals
        last_close = last_row["Close"]
        fib_382 = last_row["Fib_0.382"]
        fib_618 = last_row["Fib_0.618"]

        # Default to hold (0) for all tickers
        qty = 0  

        # Ensure we have valid Fibonacci levels
        if not pd.isna(fib_382) and not pd.isna(fib_618) and last_close > 0:
            # Adjust trade size dynamically
            max_trade_capital = capital * risk_per_trade
            num_shares = min(int(max_trade_capital / last_close), int(capital / last_close))

            # Generate buy/sell orders
            if last_close > fib_618:  # Buy signal
                qty = num_shares
            elif last_close < fib_382:  # Sell signal
                qty = -num_shares

        # Ensure every ticker gets an order (buy, sell, or hold)
        orders.append((ticker, qty))

    return orders

In [11]:
# Llamamos a calculate_orders usando el diccionario final con indicadores
orders = calculate_orders(portfolio_data)
print("\nÓrdenes generadas:", orders)


Órdenes generadas: [('AAPL', 7), ('JPM', -7), ('NFLX', -1), ('CSCO', -27), ('GE', -9), ('DHR', 8), ('ANET', -21), ('UNP', 7), ('FI', -8), ('VRTX', 3), ('NVDA', -15), ('LLY', -2), ('BAC', -42), ('MS', -14), ('AMD', -17), ('INTU', 2), ('C', -25), ('UBER', 23), ('MU', -19), ('MRVL', -25), ('MSFT', -4), ('V', -5), ('JNJ', 10), ('NOW', -2), ('ABT', 12), ('MRK', 18)]


In [15]:
# ------------------------------------
# Envío de órdenes utilizando doubletau_quant
# ------------------------------------
bot = BotConnector(token="1ae620573a5c2c01")
MAX_OPERACIONES_POR_TICKER = 10
operation_counter = defaultdict(int)

print("\nEnviando órdenes...")
for ticker, qty in orders:
    if operation_counter[ticker] < MAX_OPERACIONES_POR_TICKER:
        operation_counter[ticker] += 1
        print(f"Enviando orden para {ticker}: {qty} acción(es)")
        bot.send_order(ticker, qty)
    else:
        print(f"No se pueden enviar más órdenes para {ticker}. Límite alcanzado.")

print("\nProceso completo.")


Enviando órdenes...
Enviando orden para AAPL: 7 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para JPM: -7 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para NFLX: -1 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para CSCO: -27 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para GE: -9 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para DHR: 8 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para ANET: -21 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para UNP: 7 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para FI: -8 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para VRTX: 3 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para NVDA: -15 acción(es)
Error al enviar operación: Mercado cerrado
Enviando orden para LLY: -2 acción(es)
Error al enviar operación: Mercado 